In [1]:
# import required libraries
import pandas as pd
import yfinance as yf
import numpy as np
import math 

In [2]:
# initializing 
ticker = 'TQQQ'
start_date = '2009-01-01'
end_date = '2019-01-01'

df = yf.download(ticker, start_date, end_date, threads= False)
df = df.reset_index()
df.head()

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-02-11,0.813750,0.869792,0.811146,0.865104,0.862942,1728000
1,2010-02-12,0.841563,0.876146,0.836667,0.868646,0.866475,4300800
2,2010-02-16,0.889063,0.904375,0.875104,0.902292,0.900037,4809600
3,2010-02-17,0.914375,0.917813,0.900625,0.917604,0.915311,9590400
4,2010-02-18,0.916667,0.940521,0.911146,0.935000,0.932663,19430400


In [3]:
# parameter setup (default values in the original indicator)
length = 20
mult = 2
length_KC = 20
mult_KC = 1.5

In [4]:
# calculate Bollinger Bands
# moving average
m_avg = df['Close'].rolling(window=length).mean()
# standard deviation
m_std = df['Close'].rolling(window=length).std(ddof=0)
# upper Bollinger Bands
df['upper_BB'] = m_avg + mult * m_std
# lower Bollinger Bands 
df['lower_BB'] = m_avg - mult * m_std

In [5]:
# calculate Keltner Channel
# first we need to calculate True Range
df['tr0'] = abs(df["High"] - df["Low"])
df['tr1'] = abs(df["High"] - df["Close"].shift())
df['tr2'] = abs(df["Low"] - df["Close"].shift())
df['tr'] = df[['tr0', 'tr1', 'tr2']].max(axis=1)
# moving average of the TR
range_ma = df['tr'].rolling(window=length_KC).mean()
# upper Keltner Channel
df['upper_KC'] = m_avg + range_ma * mult_KC
# lower Keltner Channel
df['lower_KC'] = m_avg - range_ma * mult_KC

In [6]:
# check for 'squeeze'
df['squeeze_on'] = (df['lower_BB'] > df['lower_KC']) & (df['upper_BB'] < df['upper_KC'])
df['squeeze_off'] = (df['lower_BB'] < df['lower_KC']) & (df['upper_BB'] > df['upper_KC'])

In [7]:
# calculate momentum value***
highest = df['High'].rolling(window = length_KC).max()
lowest = df['Low'].rolling(window = length_KC).min()
m1 = (highest + lowest) / 2
df['value'] = (df['Close'] - (m1 + m_avg)/2)
fit_y = np.array(range(0,length_KC))
df['value'] = df['value'].rolling(window = length_KC).apply(lambda x : np.polyfit(fit_y, x, 1)[0] * (length_KC-1) + np.polyfit(fit_y, x, 1)[1], raw=True)

In [8]:
# making sense of momentum value***
df['change'] = df['value'].diff()
df['rate of change'] = round(abs(df['change'] / df['value'].shift() * 100))
df

,Date,Open,High,Low,Close,Adj Close,Volume,upper_BB,lower_BB,tr0,tr1,tr2,tr,upper_KC,lower_KC,squeeze_on,squeeze_off,value,change,rate of change
0,2010-02-11,0.813750,0.869792,0.811146,0.865104,0.862942,1728000,NaN,NaN,0.058646,NaN,NaN,0.058646,NaN,NaN,False,False,NaN,NaN,NaN
1,2010-02-12,0.841563,0.876146,0.836667,0.868646,0.866475,4300800,NaN,NaN,0.039479,0.011042,0.028437,0.039479,NaN,NaN,False,False,NaN,NaN,NaN
2,2010-02-16,0.889063,0.904375,0.875104,0.902292,0.900037,4809600,NaN,NaN,0.029271,0.035729,0.006458,0.035729,NaN,NaN,False,False,NaN,NaN,NaN
3,2010-02-17,0.914375,0.917813,0.900625,0.917604,0.915311,9590400,NaN,NaN,0.017188,0.015521,0.001667,0.017188,NaN,NaN,False,False,NaN,NaN,NaN
4,2010-02-18,0.916667,0.940521,0.911146,0.935000,0.932663,19430400,NaN,NaN,0.029375,0.022917,0.006458,0.029375,NaN,NaN,False,False,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2232,2018-12-24,16.000000,16.645000,15.160000,15.195000,15.163633,43900200,27.779867,16.110133,1.485001,0.235001,1.250000,1.485001,24.701625,19.188375,False,True,-4.296914,-1.068061,33.0
2233,2018-12-26,15.650000,17.969999,15.320000,17.930000,17.917162,94558800,27.787448,15.630552,2.650000,2.775000,0.125000,2.775000,24.561625,18.856375,False,True,-4.643566,-0.346652,8.0
2234,2018-12-27,17.299999,18.200001,16.059999,18.184999,18.171978,85458600,27.713691,15.234309,2.140001,0.270000,1.870001,2.140001,24.403500,18.544500,False,True,-4.874562,-0.230997,5.0
2235,2018-12-28,18.500000,19.055000,17.684999,18.110001,18.097033,76210600,27.303319,14.950181,1.370001,0.870001,0.500000,1.370001,23.994375,18.259125,False,True,-4.852144,0.022418,0.0


In [9]:
# entry point for long TQQQ:
# when value increases and is in negative range
df['long_tqqq'] = (df['change'] >= 0) & (df['change'].shift() <= 0) & (df['value'] < 0)

# exit point for TQQQ:
# when value decreases more than 50%
df['exit_tqqq'] = (df['rate of change'] >= 50)


# entry point for long SQQQ:
# when value decreases
df['long_sqqq'] = (df['rate of change'] >= 50) & (df['change'] > 0)

# exit point for SQQQ:
df['exit_sqqq'] = (df['rate of change'] < 10)

In [10]:
df = df.dropna()
df

,Date,Open,High,Low,Close,Adj Close,Volume,upper_BB,lower_BB,tr0,...,lower_KC,squeeze_on,squeeze_off,value,change,rate of change,long_tqqq,exit_tqqq,long_sqqq,exit_sqqq
39,2010-04-09,1.200417,1.220938,1.191146,1.218021,1.214977,14524800,1.222061,1.082887,0.029792,...,1.099177,False,True,0.052226,0.000348,1.0,False,False,False,True
40,2010-04-12,1.222188,1.232292,1.214167,1.222917,1.219861,12201600,1.230071,1.087336,0.018125,...,1.105711,False,True,0.052112,-0.000114,0.0,False,False,False,True
41,2010-04-13,1.218958,1.243333,1.207917,1.241354,1.238252,16809600,1.239127,1.093321,0.035416,...,1.112880,False,True,0.051797,-0.000316,1.0,False,False,False,True
42,2010-04-14,1.259063,1.286146,1.256458,1.285104,1.281893,15273600,1.260005,1.089797,0.029688,...,1.120034,False,True,0.058439,0.006642,13.0,False,False,False,False
43,2010-04-15,1.283750,1.307083,1.282604,1.302292,1.299037,11443200,1.281696,1.086408,0.024479,...,1.129232,False,True,0.066412,0.007973,14.0,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2232,2018-12-24,16.000000,16.645000,15.160000,15.195000,15.163633,43900200,27.779867,16.110133,1.485001,...,19.188375,False,True,-4.296914,-1.068061,33.0,False,False,False,False
2233,2018-12-26,15.650000,17.969999,15.320000,17.930000,17.917162,94558800,27.787448,15.630552,2.650000,...,18.856375,False,True,-4.643566,-0.346652,8.0,False,False,False,True
2234,2018-12-27,17.299999,18.200001,16.059999,18.184999,18.171978,85458600,27.713691,15.234309,2.140001,...,18.544500,False,True,-4.874562,-0.230997,5.0,False,False,False,True
2235,2018-12-28,18.500000,19.055000,17.684999,18.110001,18.097033,76210600,27.303319,14.950181,1.370001,...,18.259125,False,True,-4.852144,0.022418,0.0,True,False,False,True


In [26]:
capital = 10
start_long = capital
start_short = 0
long_fund = start_long
short_fund = start_short
Trades = []

In [27]:
for i in range(len(df)):
    if i%5 == 0:
        trade = []
        long_fund = 1
        gain_abs = df['Open'].iloc[len(df)-1] - df['Open'].iloc[i+1]
        gain_per = round(gain_abs / df['Open'].iloc[i+1] * 100,1)
        long_fund = round(long_fund * (100 + gain_per)/100,2)
        trade.append("Long")
        trade.append(gain_per)
        trade.append(long_fund)
        trade.append(df['Date'].iloc[i+1])
        trade.append(df['Date'].iloc[len(df)-1])
        Trades.append(trade)


In [28]:
for x in range(len(Trades)):
        print(Trades[x])

['Long', 1428.4, 15.28, Timestamp('2010-04-12 00:00:00'), Timestamp('2018-12-31 00:00:00')]
['Long', 1401.4, 15.01, Timestamp('2010-04-19 00:00:00'), Timestamp('2018-12-31 00:00:00')]
['Long', 1302.0, 14.02, Timestamp('2010-04-26 00:00:00'), Timestamp('2018-12-31 00:00:00')]
['Long', 1401.9, 15.02, Timestamp('2010-05-03 00:00:00'), Timestamp('2018-12-31 00:00:00')]
['Long', 1628.5, 17.28, Timestamp('2010-05-10 00:00:00'), Timestamp('2018-12-31 00:00:00')]
['Long', 1677.8, 17.78, Timestamp('2010-05-17 00:00:00'), Timestamp('2018-12-31 00:00:00')]
['Long', 1976.8, 20.77, Timestamp('2010-05-24 00:00:00'), Timestamp('2018-12-31 00:00:00')]
['Long', 1890.5, 19.9, Timestamp('2010-06-01 00:00:00'), Timestamp('2018-12-31 00:00:00')]
['Long', 2057.2, 21.57, Timestamp('2010-06-08 00:00:00'), Timestamp('2018-12-31 00:00:00')]
['Long', 1876.5, 19.76, Timestamp('2010-06-15 00:00:00'), Timestamp('2018-12-31 00:00:00')]
['Long', 1740.0, 18.4, Timestamp('2010-06-22 00:00:00'), Timestamp('2018-12-31 00

In [29]:
print("ticker: ", ticker)
print("start date: ", Trades[0][3])
print("end date: ", Trades[len(Trades)-1][3])
total_returns = 0
capital = 0
for x in range(len(Trades)):
        total_returns += Trades[x][2]
        capital += 1
print("total starting capital: ", capital)
print("long fund: ", start_long)
print("short fund: ", start_short)
# total_returns = round(long_fund + short_fund,2)

print("final capital: ", total_returns)
print("final long: ", long_fund)
print("final short: ", short_fund)
pnl = round(long_fund + short_fund - capital,2)
print("pnl: ", pnl)
print("pnl%: ", round((pnl/capital * 100),2))

years = (Trades[len(Trades)-1][3] - Trades[0][3]).days/365
annualized = round((pow((total_returns / capital),1/years)-1)*100,2)
print("annualized%: ", annualized)

ticker:  TQQQ
start date:  2010-04-12 00:00:00
end date:  2018-12-28 00:00:00
total starting capital:  440
long fund:  10
short fund:  0
final capital:  2440.8500000000035
final long:  1.01
final short:  0
pnl:  -438.99
pnl%:  -99.77
annualized%:  21.72


In [129]:
win_counts = 0
lose_counts = 0

for x in range(len(Trades)):
    if(Trades[x][1] > 0):
        win_counts += 1
    else:
        lose_counts += 1
        
print("win_counts: ", win_counts, "lose_counts: ", lose_counts)

win_counts:  141 lose_counts:  149


In [113]:
short_win_counts = 0
short_lose_counts = 0

for x in range(len(Trades)):
    if (Trades[x][0] == 'Short'):
        if(Trades[x][1] > 0):
            short_win_counts += 1
        else:
            short_lose_counts += 1
        
print("short_win_counts: ", short_win_counts, "short_lose_counts: ", short_lose_counts)

short_win_counts:  164 short_lose_counts:  235
